In [13]:
import pandas as pd
import pickle

In [4]:
df = pd.read_csv("../DataSet/douban_boutique/movie.csv", encoding='gbk')
df.head()

,title,types,score,director,writer,actor,account,country,label,flim_version
0,这个杀手不太冷,"剧情, 动作, 犯罪",9.4,吕克·贝松 Luc Besson,吕克·贝松 Luc Besson,"让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔, 迈克尔·巴达鲁科...",1088567,法国,1023.25298,2
1,七武士,"动作, 冒险, 剧情",9.2,黑泽明 Akira Kurosawa,"黑泽明 Akira Kurosawa, 小国英雄 Hideo Oguni, 桥本忍 Shin...","三船敏郎, 志村乔, 稻叶义男, 宫口精二, 千秋实, 加东大介, 木村功",91163,日本,83.86996,6
2,蝙蝠侠：黑暗骑士,"剧情, 动作, 科幻, 犯罪, 惊悚",9.1,克里斯托弗·诺兰 Christopher Nolan,"乔纳森·诺兰 Jonathan Nolan, 克里斯托弗·诺兰 Christopher No...","克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔, 加里·...",423487,"美国, 英国",385.37317,1
3,指环王3：王者无敌,"剧情, 动作, 奇幻, 冒险",9.1,彼得·杰克逊 Peter Jackson,"弗兰·威尔士 Fran Walsh, Philippa Boyens, 彼得·杰克逊 Pet...","维果·莫腾森, 伊莱贾·伍德, 西恩·奥斯汀, 丽芙·泰勒, 伊恩·麦克莱恩, 奥兰多·布鲁...",350209,"美国, 新西兰",318.69019,3
4,搏击俱乐部,"剧情, 动作, 悬疑, 惊悚",9.0,大卫·芬奇 David Fincher,"恰克·帕拉尼克 Chuck Palahniuk, Jim Uhls","爱德华·诺顿, 布拉德·皮特, 海伦娜·伯翰·卡特, 扎克·格雷尼尔, 米特·洛夫, 杰瑞德...",465674,"美国, 德国",419.10660,1


### 生成types集合

In [12]:
type_set = set()
for i in range(df.shape[0]):
    type_list = df.iloc[i,1].split(', ')
    for i in range(len(type_list)):
        type_list[i] = type_list[i].strip("\'")
    type_set = type_set | set(type_list)
print(type_set)

{'灾难', '战争', '爱情', 'Adult', '古装', '悬念', '西部', '情色', '悬疑', '动画', '历史', 'Reality-TV', '运动', '惊悚', '音乐', '剧情', '传记', '儿童', '奇幻', '科幻', '动作', '恐怖', '歌舞', '犯罪', '家庭', '黑色电影', '同性', '武侠', '喜剧', '惊栗', '鬼怪', '冒险'}


In [15]:
with open('type_set.pkl','wb') as f:
    pickle.dump(type_set,f)